In [1]:
#import  RNAseqQueryingInit
##static params
baseDir='/home/ec2-user/efs/all_seq/rnaseq_merged/' #Base directory

import warnings
warnings.filterwarnings('ignore')
import ipywidgets as widgets
import os
import pandas as pd
import re

In [2]:
#exampleQuery='B-Cell,T-Cell' 

In [3]:
baseDir_FnameS=pd.Series(os.listdir(baseDir))
speciesWithReprocessedData=baseDir_FnameS[baseDir_FnameS.str.contains('.npy$')].str.split('.').str[0].unique()


## Data loading

### load in SRS biospecieman annotations

In [4]:
import pandas as pd
import numpy as np

allSRS_pickle_dir='~/efs/all_seq/meta_data/allSRS.with_processed_data.flat.pickle.gz'
allSRS=pd.read_pickle(allSRS_pickle_dir)
allSRS.index.names=['SRS']

### load in technical metadata

In [5]:
sra_dump_pickle_dir='~/efs/all_seq/meta_data/sra_dump.fastqc.bowtie_algn.pickle'
%time technical_meta_data_df=pd.read_pickle(sra_dump_pickle_dir)
technical_meta_data_df[('SRAmeta','Run')]=technical_meta_data_df.index

CPU times: user 4.13 s, sys: 1.01 s, total: 5.13 s
Wall time: 5.14 s


### load the expression matrix

Check files in baseDir directory for more species

In [6]:
expression_metric='tpm' #
queryLabel='queryLabel'

def loadDf(fname,mmap_mode='r'):
    with open(fname+'.index.txt') as f:
        myIndex=map(lambda s:s.replace("\n",""), f.readlines())
    with open(fname+'.columns.txt') as f:
        myColumns=map(lambda s:s.replace("\n",""), f.readlines())
    tmpMatrix=np.load(fname+".npy",mmap_mode=mmap_mode)
    tmpDf=pd.DataFrame(tmpMatrix,index=myIndex,columns=myColumns)
    tmpDf.columns.name='Run'
    return tmpDf


### define layout

In [7]:
def returnDesignDf(queryLabelToRegexDict):
    myL=[]
    for  queryRegex in queryLabelToRegexDict.values():
        hitSrsS=allSRS[allSRS.str.contains(queryRegex,case=False)]
        myL.append(hitSrsS)

    queryLabel='queryLabel'
    mergeS=pd.concat(myL,keys=queryLabelToRegexDict.keys(),names=[queryLabel])
    mergeS_noDup=mergeS.groupby(['SRS','queryLabel']).first()
    unqiueHitMask=mergeS_noDup.groupby('SRS').size()==1
    unqiueHitSrs=unqiueHitMask.index[unqiueHitMask]
    mergeS_noDup_unique=mergeS_noDup[mergeS_noDup.index.get_level_values('SRS').isin(unqiueHitSrs)]

    #Number of SRS per query class
    mergeS_noDup_unique.groupby(queryLabel).size()

    srsToClasses_all=mergeS_noDup_unique.reset_index().set_index(['SRS'])['queryLabel']

    srsToClasses=srsToClasses_all

    ### map SRS Ids to SRR Ids

    m_SRAMeta=technical_meta_data_df[('SRAmeta','Sample')].isin(srsToClasses.index)
    technical_meta_data_df_hit=technical_meta_data_df[m_SRAMeta]

    SRAMetasrsCorrespondingQuery=srsToClasses.loc[technical_meta_data_df_hit[('SRAmeta','Sample')]].values
    technical_meta_data_df_hit[('SRAmeta',queryLabel)]=SRAMetasrsCorrespondingQuery
    relevantMetaColsL=[('SRAmeta',queryLabel),('SRAmeta','Study'),('SRAmeta','Sample'),('SRAmeta','Run'),('SRAmeta','ScientificName')]
    technical_meta_data_df_sub=technical_meta_data_df_hit[relevantMetaColsL]
    designDf=technical_meta_data_df_sub['SRAmeta']
    
    hitSrsAllAnnotS=allSRS[allSRS.index.get_level_values('SRS').isin(mergeS.index.get_level_values('SRS'))]
    srsToTextS=hitSrsAllAnnotS
    srsToTextS=pd.Series(data="NCBI SRA SRS:"+srsToTextS.index+' <br> '+srsToTextS.values,index=srsToTextS.index)
    designDf['Description']=srsToTextS[designDf.Sample].values
    return designDf
#Top species with # of reprocessed profiles

### define call back functions

In [8]:
import dash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
print(dcc.__version__) # 0.6.0 or above is required


import flask
import pandas as pd
import time
import os
from flask_caching import Cache

server = flask.Flask('app')
server.secret_key = os.environ.get('secret_key', 'secret')

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash('app', server=server, external_stylesheets=external_stylesheets)
cache = Cache(app.server, config={
    'CACHE_TYPE': 'redis',
    'CACHE_TYPE': 'filesystem',
    'CACHE_DIR': 'cache-directory',
    'CACHE_THRESHOLD': 200
})
app.title = 'Skymap'

app.config.suppress_callback_exceptions = True
app.scripts.config.serve_locally = False
dcc._js_dist[0]['external_url'] = 'https://cdn.plot.ly/plotly-basic-latest.min.js'


0.43.0


In [9]:

"""
input: search text
output:
"""


#returning the expressinon matrix based on the query: 
@cache.memoize()
def query_and_serialize_data(interact_value):
    querySpecie='Homo_sapiens'#widget_specie.get_interact_value()
    queryStr=interact_value
    listOfQueries=re.split(" *, *", queryStr)
    if len(queryStr)<3:
        raise ValueError('Please provide a query with more than 3 characters')
    queryLabelToRegexDict=dict(zip(listOfQueries,listOfQueries))
    designDf=returnDesignDf(queryLabelToRegexDict)

    #Subset the set of reprocessed data
    data_matrix_dir=baseDir+'/{specie}.gene_symbol.{expression_metric}'.format(specie=querySpecie,
                                        expression_metric=expression_metric)

    rnaseqDf=loadDf(data_matrix_dir)
    designDf_specie=designDf[(designDf['ScientificName']==querySpecie)&(designDf.Run.isin(rnaseqDf.columns))]
    queryDesignDf=designDf_specie
    #print ('Number of samples per query class that have data reprocessed in SkyMap: ',designDf_specie.groupby(queryLabel).size())
    hitDf=pd.DataFrame( list(map( lambda srrId: rnaseqDf[srrId],queryDesignDf.Run))).T
    hitDf.columns=queryDesignDf.set_index(queryDesignDf.columns.tolist()).index
    return hitDf


In [11]:
app.layout = html.Div([
#     html.Div([
#         dcc.Input(id='input-box', type='text'),
#         html.Button('Search', id='searchButton')
#     ], style={'padding-top': '5%', 'padding-left': '5%'}),
#     dcc.Graph(id='my-graph') ,
#     html.Div(id='output-container-button',
#              children='Enter a value and press submit'),
    
#     html.Div(id='designDf', style={'display': 'none'})
    dcc.Location(id='url', refresh=False),
    html.Div([
        dcc.Input(id='input-box', type='text'),
        html.Button('Search', id='searchButton')
    ], style={'padding-top': '5%', 'padding-left': '5%'}),
    html.Div(id='output-container-button',
             children='Enter a value and press submit'),
    html.Div(id='page-content')
])

In [12]:
index_page = html.Div([
    dcc.Link('Go to PCA Plot', href='/pca-plot'),
    html.Br(),
    dcc.Link('Go to Volcano Plot', href='/volcano-plot')
])

"""
,
    [dash.dependencies.State('input-box', 'value')]
"""

@app.callback(
    dash.dependencies.Output('output-container-button', 'children'),
    [dash.dependencies.Input('searchButton', 'n_clicks'),
    # Input('session-id', 'children')
    ],
    [dash.dependencies.State('input-box', 'value')])
def searchFunction(n_clicks,interact_value):
    if n_clicks: #if n_clicks not null
        #gnerate and savee expression matrix
        hitDf=query_and_serialize_data(interact_value)
        return "Number of sequencing experiment returned: {}".format(hitDf.shape[1])

In [13]:
pca_page_layout = html.Div([
    html.H1('PCA Plot'),
    html.Br(),
    dcc.Link('Go to Volcano Plot', href='/volcano-plot'),
    html.Br(),
    dcc.Link('Go back to home', href='/'),
    html.Br(),
    dcc.Graph(id='my-graph-pca'),
    html.Div(id='designDf-pca', style={'display': 'none'})
])


@app.callback(
    dash.dependencies.Output('my-graph-pca','figure'),
    [dash.dependencies.Input('output-container-button','children'),
     #dash.dependencies.Input('input-box', 'value'),
    ],[dash.dependencies.State('input-box', 'value')])
def plotPCA(container,interact_value):
        print ('called plot PCA: ',container,interact_value)
        #if str( "Number of sequencing experiment returned") in str(container):
        #    print ('in loop')
        if len(str(interact_value))>0:
            interact_value=interact_value

            import numpy as np
            hitDf=query_and_serialize_data(interact_value)
            inputAnalyzeDf=np.log2(hitDf+1)
            inPcaDf=inputAnalyzeDf.T

            from sklearn import decomposition
            import plotly.graph_objs as go
            #from plotly.offline import iplot, init_notebook_mode

            PCA=decomposition.PCA(n_components=3)

            pcaM=PCA.fit_transform((inPcaDf))
            pcaDf=pd.DataFrame( data=pcaM,index=inPcaDf.index)

            layout_3d = go.Layout(
                        scene = dict(
                        xaxis = dict(
                            title='PC0'),
                        yaxis = dict(
                            title='PC1',),
                        zaxis = dict(
                            title='PC2',),),
                      )

            #fig = go.Figure(layout=layout_3d)
            dataL=[]
            for label, sub_pca_df in pcaDf.groupby('queryLabel'):
                dataL.append(go.Scatter3d(x=sub_pca_df[0], y=sub_pca_df[1],z=sub_pca_df[2],
                                  name=label,
                                hovertext=sub_pca_df.index.get_level_values('Description'),
                                mode = 'markers')
                            )

            return {'data':dataL,'layout':layout_3d}

In [14]:
volcano_page_layout = html.Div([
    html.H1('Volcano Plot'),
    html.Br(),
    dcc.Link('Go to PCA Plot', href='/pca-plot'),
    html.Br(),
    dcc.Link('Go back to home', href='/'),
    html.Br(),
    dcc.Graph(id='my-graph-volcano') ,
    html.Div(id='designDf', style={'display': 'none'})
])

@app.callback(
    dash.dependencies.Output('my-graph-volcano','figure'),
    [dash.dependencies.Input('output-container-button','children'),
     #dash.dependencies.Input('input-box', 'value'),
    ],[dash.dependencies.State('input-box', 'value')])
def plotVolcano(container,interact_value):
        print ('called plot Volcano: ',container,interact_value)
        #if str( "Number of sequencing experiment returned") in str(container):
        #    print ('in loop')
        if len(str(interact_value))>0:
            interact_value=interact_value

            import numpy as np
            hitDf=query_and_serialize_data(interact_value)
            inputAnalyzeDf=np.log2(hitDf+1)
            listOfQueries=list(inputAnalyzeDf.columns.get_level_values('queryLabel').unique())
            inPcaDf=inputAnalyzeDf.T
            
            from sklearn import decomposition
            import plotly.graph_objs as go
            from scipy import stats
            
            expression_effect_size_filter = 0.1 
            infImputation = 200
            labelA = listOfQueries[0]
            labelB = listOfQueries[1]
            
            t,p = stats.ttest_ind(inputAnalyzeDf[labelA],inputAnalyzeDf[labelB],axis=1)
            effectDiff = inputAnalyzeDf[labelA].mean(axis=1)-inputAnalyzeDf[labelB].mean(axis=1)
            effectLabel = 'expression of : "{}" - "{}"'.format(labelA, labelB)
            
            tmpDf = pd.DataFrame({'t':t,'-log10(p)':-np.log10(p),effectLabel:effectDiff,'u':inputAnalyzeDf.mean(axis=1)},index=inputAnalyzeDf.index)
            tmpDf.loc[tmpDf['-log10(p)']==-np.inf,'-log10(p)'] = -infImputation
            tmpDf.loc[tmpDf['-log10(p)']==np.inf,'-log10(p)'] = infImputation

            plotDf = tmpDf[tmpDf['u']>=expression_effect_size_filter].dropna()
            yLabel = '-log10(p)'
            xLabel = effectLabel
            
            layout = go.Layout(
                yaxis={'title':"-log10(p)"},
                xaxis={'title':"{} - {}".format(labelA,labelB)}
            )
            
            #fig = go.Figure(layout=layout_3d)
            dataL=[]
            dataL.append(go.Scatter(
                            x=plotDf[xLabel],
                            y=plotDf[yLabel],
                            mode='markers',
                            hovertext=plotDf.index.values,
                            name=''
                        )
            )

            return {'data':dataL,'layout':layout}

In [15]:
@app.callback(dash.dependencies.Output('page-content', 'children'),
              [dash.dependencies.Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/pca-plot':
        return pca_page_layout
    elif pathname == '/volcano-plot':
        return volcano_page_layout
    else:
        return index_page


In [ ]:
app.run_server()

 * Serving Flask app "app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Feb/2019 21:02:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2019 21:02:14] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2019 21:02:15] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2019 21:02:15] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2019 21:02:15] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2019 21:02:15] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2019 21:02:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2019 21:02:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2019 21:02:54] "POST /_dash-update-component HTTP/1.1" 200 -


called plot Volcano:  Number of sequencing experiment returned: 1676 T-Cell,B-Cell


127.0.0.1 - - [26/Feb/2019 21:03:00] "POST /_dash-update-component HTTP/1.1" 200 -


### scratch

B-Cell,T-Cell

In [ ]:
#!conda install -y git

In [ ]:
#!git init

In [ ]:
#!git remote add origin https://github.com/brianyiktaktsui/skymap_web_server.git

In [ ]:
!git add queryToData.ipynb 

In [ ]:
!git commit -m "Add volcano plot"


In [ ]:
!git push -u origin master